<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Prompt Engineering for Decisions

Decision Intelligence applied in this module:  
* Optimizing prompts for logic, reasoning and decisions
* Examples of applying decision intelligence techniques in prompts and their output effect 
* Prompt techniques such as Chain of Thought to improve the quality of the decision reasoning and outcome 


In this module, the Semantic Kernel ability to chat completion experience will be used to optimize system prompts (personas) and instructive prompts. No new specific Semantic Kernel functionality will be introduced. This module will focus on introducing decision intelligence prompting techniques.  

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [48]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 8.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 8.0.0"

using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0

In [49]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 8.0.0"
#r "nuget: Microsoft.SemanticKernel, 1.20.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 8.0.0 Microsoft.SemanticKernel, 1.20.0

Using Azure OpenAI Service


### Step 2 - Chain of Thought

> "The happiness of your life depends upon the quality of your thoughts." 
>
> -- <cite>Marcus Aurelius (Roman emperor, philosopher)</cite>  

One of the most effective techniques to improve logic, reasoning and overall decision-making for most LLMs is to use "Chain of Thought" (CoT) techniques. The "Chain of Thought" approach helps by breaking down complex tasks into smaller steps, making it easier to solve problems without missing important details. It improves accuracy because each step is made clear to the LLM and helps avoid mistakes. This method also makes the thought process easier to understand and explain, allowing for easier corrections when something goes wrong. The approach is useful for handling difficult concepts and multi-step problems like math or logic, making things clearer and more manageable.

The name of the "Chain of Thought" technique has been recently popularized by Generative AI. However, this technique is not new nor is it specific only to Generative AI models. Breaking down complex tasks into simpler more approachable steps and organizing thoughts has been used by decision-makers, professional services organizations and management consulting companies for quite some time. Prior to Generative AI, the concepts have been known as: "structured thinking", "MECE framework" (Mutually Exclusive, Collectively Exhaustive), and "hypothesis-driven problem-solving".  

Approach the problem with a simple prompt techniques. The system prompt will be set to basic & clear decision assistant persona instructions.  

In [65]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a decision intelligence assistant. 
Provide structured, logical, and comprehensive advice.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponse = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    decisionResponse += content;
    Console.Write(content);
}

To solve this problem, the farmer needs to follow a specific sequence of steps to ensure that neither the goat nor the cabbage gets eaten. Here is a structured plan:

1. **First Trip: Take the Goat Across**
   - The farmer takes the goat across the river and leaves it on the other side.
   - The wolf and the cabbage remain on the original side.

2. **Second Trip: Return Alone**
   - The farmer returns to the original side alone.

3. **Third Trip: Take the Wolf Across**
   - The farmer takes the wolf across the river.
   - He leaves the wolf on the other side but takes the goat back with him to the original side.

4. **Fourth Trip: Take the Cabbage Across**
   - The farmer takes the cabbage across the river and leaves it with the wolf.
   - The goat remains on the original side.

5. **Fifth Trip: Return Alone**
   - The farmer returns to the original side alone.

6. **Sixth Trip: Take the Goat Across Again**
   - The farmer takes the goat across the river one final time.

By following t

Approach the same problem with a much more sophisticated "Chain of Thought" (CoT) system prompt to break the problem down and think about it more in depth. 

In [66]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptChainOfThought = """
You are a decision intelligence assistant designed to think through problems step-by-step using Chain-of-Thought (COT) prompting. 
Before providing any answer, you must: 
Understand the Problem: Carefully read and understand the user's question or request. 
Break Down the Reasoning Process: Outline the steps required to solve the problem or respond to the request logically and sequentially. Think aloud and describe each step in detail. 
Explain Each Step: Provide reasoning or calculations for each step, explaining how you arrive at each part of your answer. 
Provide structured, logical, and comprehensive advice. 
Arrive at the Final Answer: Only after completing all steps, provide the final answer or solution. 
Review the Thought Process: Double-check the reasoning for errors or gaps before finalizing your response. 
Always aim to make your thought process transparent and logical. Communicate the final decision using the Minto Pyramid Principle.
""";

// Simple instruction prompt to plan retirement
var puzzlePrompt = """
A farmer is on one side of a river with a wolf, a goat, and a cabbage. 
When he is crossing the river in a boat, he can only take one item with him at a time. 
The wolf will eat the goat if left alone together, and the goat will eat the cabbage if left alone together. 
How can the farmer transport the goat across the river without it being eaten?
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPromptChainOfThought);
chatHistory.AddUserMessage(puzzlePrompt);

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

var decisionResponseChainOfThought = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory, openAIPromptExecutionSettings))
{
    decisionResponseChainOfThought += content;
    Console.Write(content);
}

To solve this problem, we need to ensure that at no point are the wolf and goat left alone together, nor the goat and cabbage left alone together. Let's break down the steps logically:

1. **Understand the Problem:**
   - The farmer can only take one item at a time across the river.
   - The wolf cannot be left alone with the goat.
   - The goat cannot be left alone with the cabbage.

2. **Identify the Constraints:**
   - The farmer must always be present to prevent the wolf from eating the goat or the goat from eating the cabbage.

3. **Plan the Steps:**
   - We need to figure out a sequence of crossings that ensures the safety of all items.

4. **Step-by-Step Solution:**
   - **Step 1:** The farmer takes the goat across the river first.
     - **State:** Wolf and cabbage on the original side, goat on the other side.
   - **Step 2:** The farmer returns alone to the original side.
     - **State:** Wolf and cabbage on the original side, goat on the other side.
   - **Step 3:** The farm

The Generative AI model has generated two different decision approaches to the same retirement decision question. Let's use the LLM to decide which decision answer is more thought out and why. Ideally, the evaluation should be done by another family or an LLM from a different provider, not the LLM that generated the responses.  

In [67]:
var systemPromptEvaluateResponses = """
You are an assistant that is evaluating an approach response to a question.
You will be provided with an important decision as well as the proposed approach.
Score the approach on a scale of 1 to 10 based on its thought out reasoning in approaching the decision.
""";

var decisionEvaluationTemplateApproach1 = $"""
Decision: {puzzlePrompt}
--------------
Approach: {decisionResponse} 
""";

var decisionEvaluationTemplateApproach2 = $"""
Decision: {puzzlePrompt}
--------------
Approach: {decisionResponseChainOfThought} 
""";

// Create a new chat history object using the new Chain of Thought system prompt
var chatHistoryApproach1 = new ChatHistory();
chatHistoryApproach1.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproach1.AddUserMessage(decisionEvaluationTemplateApproach1);

var chatHistoryApproach2 = new ChatHistory();
chatHistoryApproach2.AddSystemMessage(systemPromptEvaluateResponses);
chatHistoryApproach2.AddUserMessage(decisionEvaluationTemplateApproach2);


// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 4000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

Console.WriteLine();
Console.WriteLine();
Console.WriteLine("Approach 1 Evaluation:");
Console.WriteLine("----------------------");
var evaluationResponseApproach1 = string.Empty;
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryApproach1, openAIPromptExecutionSettings))
{
    evaluationResponseApproach1 += content;
    Console.Write(content);
}

var evaluationResponseApproach2 = string.Empty;
Console.WriteLine();
Console.WriteLine();
Console.WriteLine("Approach 2 Evaluation:");
Console.WriteLine("----------------------");
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryApproach2, openAIPromptExecutionSettings))
{
    evaluationResponseApproach2 += content;
    Console.Write(content);
}



Approach 1 Evaluation:
----------------------
Score: 10

Reasoning: The approach is well thought out and clearly articulated. It provides a step-by-step solution that ensures the safety of all items (the wolf, the goat, and the cabbage) during the transportation process. Each step is logically explained, and the sequence is summarized at the end for clarity. The approach effectively addresses the constraints of the problem and ensures that neither the goat nor the cabbage gets eaten at any point.

Approach 2 Evaluation:
----------------------
I would rate this approach a 10.

The reasoning is thorough and well-structured, addressing each constraint and ensuring that the solution adheres to the rules of the problem. The step-by-step breakdown is clear and logical, ensuring that at no point are the wolf and goat left alone together, nor the goat and cabbage left alone together. The approach also includes a review of the thought process to confirm that the solution is correct. This demo

### Step 3 - Logic Reasoning

WIP